#### just opening the netcdf `dump=True` from `Cyclone_damage_funcs.py`
#### then saving out as geotiff 

In [60]:
import glob
import numpy as np
import xarray as xr
import rioxarray
import datacube
from datacube.testutils.io import rio_slurp_xarray
from datacube.utils.cog import write_cog

In [61]:
cyclone_name = 'Yasi'

#### immediate impacts (cyclone year before and cyclone year after)

#### long term impacts

In [62]:
# all = long term impacts
nc_all = (glob.glob('./'+cyclone_name+'/*_all.nc'))
print(glob.glob('./'+cyclone_name+'/*_all.nc'))

['./Yasi/Yasi_138_-191_all.nc', './Yasi/Yasi_144_-201_all.nc', './Yasi/Yasi_144_-211_all.nc', './Yasi/Yasi_144_-191_all.nc', './Yasi/Yasi_153_-219_all.nc', './Yasi/Yasi_144_-219_all.nc']


In [67]:
for i in nc_all:
    nc_ds = xr.open_dataset(i)
    singleband_da = nc_ds.damage_level.isel(time=0)
    print(i[7:-3])
    print(np.unique(singleband_da.values))
    # Write GeoTIFF to a location
    write_cog(geo_im=singleband_da,
              fname=i[7:-3]+'.tif',
              overwrite=True)


Yasi_138_-191_all
[-1]
Yasi_144_-201_all
[-1  0  1  2  3  4  5]
Yasi_144_-211_all
[-1  0  1  2  3  4  5]
Yasi_144_-191_all
[-1  0  1  3  4  5]
Yasi_153_-219_all
[-1  0  1  2  3  4  5]
Yasi_144_-219_all
[-1  0  1  2  3  4]
